In [1]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dense, BatchNormalization
from model import yolov3
from IPython.core.display import display, HTML
from utils import center_to_minmax, iou_calculate, non_max_suppression
from utils import convert_to_original_size, load_coco_names, draw_boxes
from PIL import ImageDraw, Image

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "9"
gpus = tf.config.experimental.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(gpus[0], True)
display(HTML("<style>.container { width:98% !important;}</style>"))

In [3]:
# load image
img = Image.open("/raid/peterchen/research/TF2_yoloV3/tim.jpg")
img_resized = img.resize(size=(416, 416))
img_resized = np.array(img_resized, dtype=np.float32)
img_resized = tf.expand_dims(tf.constant(img_resized)/255., axis=0)

In [4]:
# load image
img = tf.io.read_file("/raid/peterchen/research/TF2_yoloV3/tim.jpg")
img = tf.image.decode_jpeg(img)
img = tf.image.resize(img, (416, 416), method="nearest")
img = tf.cast(img, tf.float32)
img /= 255.
img_resized = tf.expand_dims(img, axis=0)

In [5]:
# Build Model
yolov3_model = yolov3.model()
yolov3.load_weights("weights/yolov3.weights", yolov3_model)

In [6]:
model_output_1, model_output_2, model_output_3 = yolov3_model(img_resized)

UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above. [Op:Conv2D]

In [ ]:
print(model_output_1.shape)
print(model_output_2.shape)
print(model_output_3.shape)

In [ ]:
ANCHORS = [(10, 13), (16, 30), (33, 23), (30, 61), (62, 45), (59, 119), (116, 90), (159, 198), (373, 326)]

In [ ]:
predictions_1 = yolov3.detection_layer(model_output_1, (416, 416), ANCHORS[6:9], num_classes=80)
predictions_2 = yolov3.detection_layer(model_output_2, (416, 416), ANCHORS[3:6], num_classes=80)
predictions_3 = yolov3.detection_layer(model_output_3, (416, 416), ANCHORS[0:3], num_classes=80)

In [ ]:
print(predictions_1.shape)
print(predictions_2.shape)
print(predictions_3.shape)

In [ ]:
detections = tf.concat([predictions_1, predictions_2, predictions_3], axis=1)

In [ ]:
boxes = center_to_minmax(detections)

In [ ]:
conf_threshold = 0.5
iou_threshold = 0.4
class_names = "coco.names"

In [ ]:
filtered_boxes = non_max_suppression(boxes, confidence_threshold=conf_threshold, iou_threshold=iou_threshold)

In [ ]:
filtered_boxes

In [ ]:
filtered_boxes

In [ ]:
classes = load_coco_names(class_names)

In [ ]:
draw_boxes(filtered_boxes, img, classes, (416, 416))

In [ ]:
img.save("bbb.jpg")

In [ ]:
plt.imshow(img)